In [ ]:
##Create a network table (as the import for subsequent visualization in Cytoscape)
#Nodes = event types
#Edges = indicate which events are connected based on the sequence of time stamps
#Edges can have values attached: 
  #In this case, either (1) aggregated number of connections over certain period of time (e.g. counts),
    #or (2) the time difference between the event timestamps (e.g. seconds)

In [3]:
#connect to Redshift database:
%load_ext sql
%config SqlMagic.displaylimit = 100
%config SqlMagic.autopandas= True
connection_string = open("connection","r").read()
%sql $connection_string

u'Connected: testuser@dev'

In [4]:
#for this example: select just one day's worth of data:
#all the entries from eventsx from March 26th
test = %sql SELECT keyboardid, eventname,timestamp FROM datatest.eventsx WHERE DATE(timestamp) = '2015-03-26' ORDER BY keyboardid, timestamp

176570 rows affected.


In [5]:
#import:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
##take a quick look at data:

In [6]:
#how many unique users?
users = test['keyboardid'].unique()
len(users)

16625

In [7]:
#what are the different event types? (e.g. nodes)
e = test['eventname'].unique()
e

array([u'andrfbm_open', u'andrfbm_browsetag', u'andrfbm_listview_send',
       u'andrfbm_itemview', u'andrfbm_itemview_reply', u'andrfbm_search',
       u'andrfbm_fbmreply', u'andrfbm_listview_reply',
       u'andrfbm_itemview_send', u'andrfbm_itemview_browsetag'], dtype=object)

In [8]:
#group data by person:
grouped = test.groupby('keyboardid')

In [9]:
#defining sessions by '_open' events: 
session = {}
for index, (name,group) in enumerate(grouped):
    sess = 0
    for idx, row in group.iterrows():
        if row['eventname'] == 'andrfbm_open':
            sess+= 1
        session[idx] = sess

In [10]:
#add sessionid column :
r, v = zip(*session.items())
session_series = pd.Series.from_array(np.array(v), index=np.array(r), name='sessionid')
test_plus_sessionid = pd.concat([test,session_series],axis=1)

In [11]:
#add eventname_next column:
eventnamecol = np.copy(test_plus_sessionid['eventname'])
eventnamecol[-1] = 'end'
eventnamecol[:-1] = eventnamecol[1:]
test_plus_sessionid['eventname_next'] = eventnamecol

In [12]:
#add timestamp_next column:
timestampcol = np.copy(test_plus_sessionid['timestamp'])
timestampcol[:-1] = timestampcol[1:]
test_plus_sessionid['timestamp_next'] = timestampcol

In [13]:
#add new column that takes time difference:
timediff = test_plus_sessionid['timestamp_next'] - test_plus_sessionid['timestamp']
test_plus_sessionid['timestampdiffsec'] = timediff
#rename dataframe to 'T':
T = test_plus_sessionid
T.head()

,keyboardid,eventname,timestamp,sessionid,eventname_next,timestamp_next,timestampdiffsec
0,Mjc0MDA0MA,andrfbm_open,2015-03-26 01:33:54,1,andrfbm_open,2015-03-26 14:15:52,12:41:58
1,Mjc0MDA0MA,andrfbm_open,2015-03-26 14:15:52,2,andrfbm_open,2015-03-26 14:15:54,00:00:02
2,Mjc0MDA0MA,andrfbm_open,2015-03-26 14:15:54,3,andrfbm_browsetag,2015-03-26 14:16:08,00:00:14
3,Mjc0MDA0MA,andrfbm_browsetag,2015-03-26 14:16:08,3,andrfbm_browsetag,2015-03-26 14:16:20,00:00:12
4,Mjc0MDA0MA,andrfbm_browsetag,2015-03-26 14:16:20,3,andrfbm_listview_send,2015-03-26 14:16:35,00:00:15


In [14]:
#Since there are no timestamps for when a user closes the app,
#we assume that when the next open event occurs, there was a close or end event before that:
#function to change (open --> open) to (open --> end)
def change_to_close(x):
    if (x['eventname'] == 'andrfbm_open') & (x['eventname_next'] == 'andrfbm_open'):
        return 'andrfbm_end'
    else:
        return x['eventname_next']

In [15]:
#run 'change_to_close' function:
S = T.copy()
S['eventname_next'] = S.apply(change_to_close,axis = 1)

In [16]:
#change timestampdiffsec to be integer seconds:
S['timestampdiffsec'] = S['timestampdiffsec'].astype('timedelta64[s]')

In [17]:
#similar to reasoning for above function, open cannot be in the next column
#function to get rid of (open) in 'eventname_next':
def remove_nextopen(x):
    if (x['eventname_next'] == 'andrfbm_open'):
        return 'andrfbm_end'
    else:
        return x['eventname_next']

In [18]:
#run 'remove_nextopen'
U = S.copy()
U['eventname_next'] = U.apply(remove_nextopen,axis = 1)

In [19]:
#function to remove the timestamps for 'eventname_next' == '..._end' because not known.
def remove_end_timestamps(x):
    if (x['eventname_next'] == 'andrfbm_end'):
        return 0
    else:
        return x['timestampdiffsec']

In [20]:
V = U.copy()
V['timestampdiffsec'] = V.apply(remove_end_timestamps,axis = 1)

In [21]:
#need to remove instances when eventname_next is not actually from the same person 
#shows up as negative values (related to having search be an 'open feature'?)
def remove_neg_timestampdiff(x):
    if (x['timestampdiffsec'] < 0) :
        return 'andrfbm_end'
    else:
        return x['eventname_next']

In [22]:
W = V.copy()
W['eventname_next'] = W.apply(remove_neg_timestampdiff,axis = 1)

In [23]:
#Now, remove the negative values
def remove_zeros(x):
    if (x['timestampdiffsec'] < 0) :
        return 0
    else:
        return x['timestampdiffsec']    

In [24]:
Y = W.copy()
Y['timestampdiffsec'] = Y.apply(remove_zeros,axis = 1)

In [26]:
#remove instances where _send & _reply events have end as their next event
#because they can be considered a different end point.
Z = Y.copy()
mask = (Z['eventname'].isin(['andrfbm_listview_send','andrfbm_itemview_reply',
                         'andrfbm_fbmreply','andrfbm_listview_reply','andrfbm_itemview_send']) & 
            (Z['eventname_next'] == 'andrfbm_end'))
AA = Z.drop(Z.index[mask])

In [27]:
#saving output to .csv for later import into Cytoscape:
BB = AA.groupby(['eventname','eventname_next'])['timestampdiffsec'].mean()
BB.to_csv('time_matrix.csv',sep=',')

In [28]:
#saving output to .csv for later import into Cytoscape:
CC = AA.groupby(['eventname','eventname_next']).count()
CC.to_csv('count_matrix.csv',sep=',')

In [1]:
#Instructions for Cytoscape:
#(0) 
#pull column three from time matrix as 'seconds' and append to count matrix &
#keep only one of the numerical columns from count_matrix as 'counts'
#save as a new .csv
#(1) Open cytoscape
#(2) File --> Import --> Network --> File (load the .csv)
#(3) Select Source Node = 'eventname'
    #Select Interaction type = Default
    #Select Target Node = 'eventname_next'
    #Click on 'counts' & 'seconds' columns so they are highlighted in blue
#(4) Change style to 'Directed'    
#(5) Customize node and edges as desired. 
#Can create separate plots with weighting of edges corresponding to 'counts' or 'seconds'